In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="b2e6b7236b5049a18bfde863c279fae9" 
secret = "13ca6fa8018d4c8eb03ff8510fc96d2c"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'k-pop'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 7.844312500000001


In [13]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 1000


In [14]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,LISA,MONEY,7hU3IHwjX150XLoTVmjD0q,92
1,BTS,Butter,2bgTY4UwhfBYhGT4HUYStN,90
2,TWICE,The Feels,1XyzcGhmO7iUamSS94XfqY,87
3,Bryson Tiller,Don't,3pXF1nA74528Edde4of9CC,81
4,BTS,Dynamite,4saklk6nie3yiGePpBwUoc,88


In [22]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 7.466742500000009


In [20]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 0


In [23]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (1000, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.831,0.554,1,-9.998,0,0.2180,0.16100,0.000061,0.1520,0.396,140.026,audio_features,7hU3IHwjX150XLoTVmjD0q,spotify:track:7hU3IHwjX150XLoTVmjD0q,https://api.spotify.com/v1/tracks/7hU3IHwjX150...,https://api.spotify.com/v1/audio-analysis/7hU3...,168228,4
1,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.000000,0.0906,0.695,109.997,audio_features,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4
2,0.808,0.903,9,-2.179,1,0.0729,0.08990,0.000000,0.0816,0.922,120.044,audio_features,1XyzcGhmO7iUamSS94XfqY,spotify:track:1XyzcGhmO7iUamSS94XfqY,https://api.spotify.com/v1/tracks/1XyzcGhmO7iU...,https://api.spotify.com/v1/audio-analysis/1Xyz...,198053,4
3,0.765,0.356,11,-5.556,0,0.1950,0.22300,0.000000,0.0963,0.189,96.991,audio_features,3pXF1nA74528Edde4of9CC,spotify:track:3pXF1nA74528Edde4of9CC,https://api.spotify.com/v1/tracks/3pXF1nA74528...,https://api.spotify.com/v1/audio-analysis/3pXF...,198293,4
4,0.746,0.765,6,-4.410,0,0.0993,0.01120,0.000000,0.0936,0.737,114.044,audio_features,4saklk6nie3yiGePpBwUoc,spotify:track:4saklk6nie3yiGePpBwUoc,https://api.spotify.com/v1/tracks/4saklk6nie3y...,https://api.spotify.com/v1/audio-analysis/4sak...,199054,4


In [25]:
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,LISA,MONEY,7hU3IHwjX150XLoTVmjD0q,92,0.831,0.554,1,-9.998,0,0.2180,...,0.000061,0.1520,0.396,140.026,audio_features,spotify:track:7hU3IHwjX150XLoTVmjD0q,https://api.spotify.com/v1/tracks/7hU3IHwjX150...,https://api.spotify.com/v1/audio-analysis/7hU3...,168228,4
1,BTS,Butter,2bgTY4UwhfBYhGT4HUYStN,90,0.759,0.459,8,-5.187,1,0.0948,...,0.000000,0.0906,0.695,109.997,audio_features,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4
2,TWICE,The Feels,1XyzcGhmO7iUamSS94XfqY,87,0.808,0.903,9,-2.179,1,0.0729,...,0.000000,0.0816,0.922,120.044,audio_features,spotify:track:1XyzcGhmO7iUamSS94XfqY,https://api.spotify.com/v1/tracks/1XyzcGhmO7iU...,https://api.spotify.com/v1/audio-analysis/1Xyz...,198053,4
3,Bryson Tiller,Don't,3pXF1nA74528Edde4of9CC,81,0.765,0.356,11,-5.556,0,0.1950,...,0.000000,0.0963,0.189,96.991,audio_features,spotify:track:3pXF1nA74528Edde4of9CC,https://api.spotify.com/v1/tracks/3pXF1nA74528...,https://api.spotify.com/v1/audio-analysis/3pXF...,198293,4
4,BTS,Dynamite,4saklk6nie3yiGePpBwUoc,88,0.746,0.765,6,-4.410,0,0.0993,...,0.000000,0.0936,0.737,114.044,audio_features,spotify:track:4saklk6nie3yiGePpBwUoc,https://api.spotify.com/v1/tracks/4saklk6nie3y...,https://api.spotify.com/v1/audio-analysis/4sak...,199054,4


In [26]:
df.to_csv('kpop_playlist.csv')

# Pop

In [34]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'pop'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df1 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df1.head()

Time to run this code (in seconds): 8.343579999999974
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.411982099999932
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Adele,Easy On Me,0gplL1WMoJ6iYaPgMCL0gX,100,0.604,0.366,5,-7.519,1,0.0282,...,0.0,0.1330,0.130,141.981,audio_features,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,https://api.spotify.com/v1/tracks/0gplL1WMoJ6i...,https://api.spotify.com/v1/audio-analysis/0gpl...,224695,4
1,Taylor Swift,All Too Well (10 Minute Version) (Taylor's Ver...,5enxwA8aAbwZbf5qCHORXi,95,0.631,0.518,0,-8.771,1,0.0303,...,0.0,0.0880,0.205,93.023,audio_features,spotify:track:5enxwA8aAbwZbf5qCHORXi,https://api.spotify.com/v1/tracks/5enxwA8aAbwZ...,https://api.spotify.com/v1/audio-analysis/5enx...,613027,4
2,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),27NovPIUIRrOZoCHxABJwK,97,0.736,0.704,3,-7.409,0,0.0615,...,0.0,0.0501,0.894,149.995,audio_features,spotify:track:27NovPIUIRrOZoCHxABJwK,https://api.spotify.com/v1/tracks/27NovPIUIRrO...,https://api.spotify.com/v1/audio-analysis/27No...,212000,4
3,Olivia Rodrigo,good 4 u,4ZtFanR9U6ndgddUvNcjcG,95,0.563,0.664,9,-5.044,1,0.1540,...,0.0,0.0849,0.688,166.928,audio_features,spotify:track:4ZtFanR9U6ndgddUvNcjcG,https://api.spotify.com/v1/tracks/4ZtFanR9U6nd...,https://api.spotify.com/v1/audio-analysis/4ZtF...,178147,4
4,Doja Cat,Need to Know,3Vi5XqYrmQgOYBajMWSvCi,93,0.664,0.609,1,-6.509,1,0.0707,...,0.0,0.0926,0.194,130.041,audio_features,spotify:track:3Vi5XqYrmQgOYBajMWSvCi,https://api.spotify.com/v1/tracks/3Vi5XqYrmQgO...,https://api.spotify.com/v1/audio-analysis/3Vi5...,210560,4


In [35]:
df1.to_csv('pop_playlist.csv')

# Rock

In [37]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'rock'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df2 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df2.head()

Time to run this code (in seconds): 7.262957000000142
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.3127270999998473
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,The Neighbourhood,Sweater Weather,2QjOHCTQ1Jl3zawyYOpxh6,90,0.612,0.807,10,-2.810,1,0.0336,...,0.017700,0.101,0.398,124.053,audio_features,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6,https://api.spotify.com/v1/tracks/2QjOHCTQ1Jl3...,https://api.spotify.com/v1/audio-analysis/2QjO...,240400,4
1,Weezer,Say It Ain't So,6VoIBz0VhCyz7OdEoRYDiA,76,0.634,0.551,3,-7.136,1,0.0352,...,0.000018,0.070,0.453,151.779,audio_features,spotify:track:6VoIBz0VhCyz7OdEoRYDiA,https://api.spotify.com/v1/tracks/6VoIBz0VhCyz...,https://api.spotify.com/v1/audio-analysis/6VoI...,258827,4
2,Imagine Dragons,Enemy (with JID) - from the series Arcane Leag...,1r9xUipOqoNwggBpENDsvJ,95,0.728,0.783,11,-4.424,0,0.2660,...,0.000000,0.434,0.555,77.011,audio_features,spotify:track:1r9xUipOqoNwggBpENDsvJ,https://api.spotify.com/v1/tracks/1r9xUipOqoNw...,https://api.spotify.com/v1/audio-analysis/1r9x...,173381,4
3,Måneskin,Beggin',3Wrjm47oTz2sjIgck11l5e,94,0.714,0.800,11,-4.808,0,0.0504,...,0.000000,0.359,0.589,134.002,audio_features,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
4,The Neighbourhood,Daddy Issues,5E30LdtzQTGqRvNd7l6kG5,87,0.588,0.521,10,-9.461,1,0.0329,...,0.149000,0.123,0.337,85.012,audio_features,spotify:track:5E30LdtzQTGqRvNd7l6kG5,https://api.spotify.com/v1/tracks/5E30LdtzQTGq...,https://api.spotify.com/v1/audio-analysis/5E30...,260173,4


In [38]:
df2.to_csv('rock_playlist.csv')

# soft rock

In [39]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'soft rock'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df3 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df3.head()

Time to run this code (in seconds): 8.74719640000012
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.5694416999999703
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Daryl Hall & John Oates,Jingle Bell Rock - Daryl's Version,6pVW5LRWgeLaHudxauOTJU,80,0.666,0.841,2,-3.592,1,0.0312,...,0.000000,0.1130,0.840,128.706,audio_features,spotify:track:6pVW5LRWgeLaHudxauOTJU,https://api.spotify.com/v1/tracks/6pVW5LRWgeLa...,https://api.spotify.com/v1/audio-analysis/6pVW...,126360,4
1,Bob Seger,Old Time Rock & Roll,5EOoMWIB9iK4ZpcSex9Ec7,74,0.714,0.937,6,-2.947,1,0.0399,...,0.000000,0.0731,0.964,124.130,audio_features,spotify:track:5EOoMWIB9iK4ZpcSex9Ec7,https://api.spotify.com/v1/tracks/5EOoMWIB9iK4...,https://api.spotify.com/v1/audio-analysis/5EOo...,194147,4
2,John Denver,Rocky Mountain High,1ne9wOtDF2jM6Cm8WBkaER,67,0.455,0.437,4,-11.411,1,0.0278,...,0.000000,0.5010,0.724,81.396,audio_features,spotify:track:1ne9wOtDF2jM6Cm8WBkaER,https://api.spotify.com/v1/tracks/1ne9wOtDF2jM...,https://api.spotify.com/v1/audio-analysis/1ne9...,283640,4
3,Steve Miller Band,Rock'n Me,1orVKbp6vqtfAPOmvRofVq,66,0.695,0.582,4,-11.350,1,0.0651,...,0.000000,0.1230,0.892,122.433,audio_features,spotify:track:1orVKbp6vqtfAPOmvRofVq,https://api.spotify.com/v1/tracks/1orVKbp6vqtf...,https://api.spotify.com/v1/audio-analysis/1orV...,187067,4
4,Survivor,Eye of the Tiger,2KH16WveTQWT6KOG9Rg6e2,77,0.817,0.599,0,-9.249,0,0.0328,...,0.000311,0.0873,0.548,108.873,audio_features,spotify:track:2KH16WveTQWT6KOG9Rg6e2,https://api.spotify.com/v1/tracks/2KH16WveTQWT...,https://api.spotify.com/v1/audio-analysis/2KH1...,245640,4


In [40]:
df3.to_csv('soft_rock_playlist.csv')

# Jazz

In [41]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'jazz'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df4 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df4.head()

Time to run this code (in seconds): 7.865940799999862
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.128195900000037
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Brenda Lee,Rockin' Around The Christmas Tree,2EjXfH91m7f8HiJN1yQg97,88,0.589,0.472,8,-8.749,1,0.0502,...,0.000000,0.505,0.898,67.196,audio_features,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4
1,Nat King Cole,The Christmas Song (Merry Christmas To You),4PS1e8f2LvuTFgUs1Cn3ON,84,0.319,0.210,1,-15.231,1,0.0341,...,0.000081,0.138,0.209,78.696,audio_features,spotify:track:4PS1e8f2LvuTFgUs1Cn3ON,https://api.spotify.com/v1/tracks/4PS1e8f2LvuT...,https://api.spotify.com/v1/audio-analysis/4PS1...,192160,4
2,Perry Como,It's Beginning to Look a Lot Like Christmas (w...,2pXpURmn6zC5ZYDMms6fwa,82,0.724,0.276,7,-12.250,1,0.0599,...,0.000000,0.116,0.718,112.874,audio_features,spotify:track:2pXpURmn6zC5ZYDMms6fwa,https://api.spotify.com/v1/tracks/2pXpURmn6zC5...,https://api.spotify.com/v1/audio-analysis/2pXp...,155933,4
3,Dean Martin,Let It Snow! Let It Snow! Let It Snow!,2uFaJJtFpPDc5Pa95XzTvg,85,0.451,0.240,1,-14.014,1,0.0351,...,0.000002,0.175,0.701,134.009,audio_features,spotify:track:2uFaJJtFpPDc5Pa95XzTvg,https://api.spotify.com/v1/tracks/2uFaJJtFpPDc...,https://api.spotify.com/v1/audio-analysis/2uFa...,117147,4
4,Michael Bublé,It's Beginning to Look a Lot like Christmas,5a1iz510sv2W9Dt1MvFd5R,86,0.339,0.214,4,-11.714,1,0.0375,...,0.000007,0.341,0.363,94.775,audio_features,spotify:track:5a1iz510sv2W9Dt1MvFd5R,https://api.spotify.com/v1/tracks/5a1iz510sv2W...,https://api.spotify.com/v1/audio-analysis/5a1i...,206640,3


In [42]:
df4.to_csv('jazz_playlist.csv')

# Indie

In [43]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'indie'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df5 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df5.head()

Time to run this code (in seconds): 7.7143857000000935
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 3.0762469999999666
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Glass Animals,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,89,0.761,0.525,11,-6.900,1,0.0944,...,0.000007,0.0921,0.531,80.870,audio_features,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://api.spotify.com/v1/tracks/3USxtqRwSYz5...,https://api.spotify.com/v1/audio-analysis/3USx...,238805,4
1,Surf Curse,Freaks,7EkWXAI1wn8Ii883ecd9xr,88,0.345,0.941,9,-9.918,1,0.0466,...,0.633000,0.0511,0.407,179.626,audio_features,spotify:track:7EkWXAI1wn8Ii883ecd9xr,https://api.spotify.com/v1/tracks/7EkWXAI1wn8I...,https://api.spotify.com/v1/audio-analysis/7EkW...,147062,4
2,Måneskin,Beggin',3Wrjm47oTz2sjIgck11l5e,94,0.714,0.800,11,-4.808,0,0.0504,...,0.000000,0.3590,0.589,134.002,audio_features,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
3,The Walters,I Love You So,4SqWKzw0CbA05TGszDgMlc,90,0.575,0.648,10,-4.891,1,0.0358,...,0.000000,0.1150,0.466,75.977,audio_features,spotify:track:4SqWKzw0CbA05TGszDgMlc,https://api.spotify.com/v1/tracks/4SqWKzw0CbA0...,https://api.spotify.com/v1/audio-analysis/4SqW...,160240,4
4,The Rare Occasions,Notion,4cHKd6qPMR1TMx55KHlida,85,0.307,0.884,9,-3.825,1,0.0813,...,0.001030,0.0849,0.314,159.579,audio_features,spotify:track:4cHKd6qPMR1TMx55KHlida,https://api.spotify.com/v1/tracks/4cHKd6qPMR1T...,https://api.spotify.com/v1/audio-analysis/4cHK...,195121,4


In [44]:
df5.to_csv('indie_playlist.csv')

# indienesia

In [50]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'indie indonesia'

for i in range(0,500,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i, tag:hipster)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df6 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df6.head()

Time to run this code (in seconds): 3.2390301000000363
(236, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.9044743000001745
Shape of the dataset: (236, 18)
Shape of the dataset: (236, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,The Mountain Goats,Indonesia* - Bonus Track,4BcxhsyI0eFbEwJHc6OgGU,24,0.489,0.381,2,-11.089,1,0.0358,...,0.002530,0.0993,0.508,131.810,audio_features,spotify:track:4BcxhsyI0eFbEwJHc6OgGU,https://api.spotify.com/v1/tracks/4BcxhsyI0eFb...,https://api.spotify.com/v1/audio-analysis/4Bcx...,136587,4
1,Amos Lee,Indonesia,78kDZ0GwxJus1GjSkAIc7q,17,0.643,0.512,11,-8.642,1,0.0308,...,0.000058,0.2740,0.308,137.977,audio_features,spotify:track:78kDZ0GwxJus1GjSkAIc7q,https://api.spotify.com/v1/tracks/78kDZ0GwxJus...,https://api.spotify.com/v1/audio-analysis/78kD...,207697,4
2,Divorce Court,Indonesia,5AHZoR2cr7KV4m6X1ZyJ1K,15,0.363,0.759,9,-6.320,1,0.0422,...,0.892000,0.1070,0.137,80.041,audio_features,spotify:track:5AHZoR2cr7KV4m6X1ZyJ1K,https://api.spotify.com/v1/tracks/5AHZoR2cr7KV...,https://api.spotify.com/v1/audio-analysis/5AHZ...,255750,4
3,Amos Lee,Indonesia,4cLcNowHHKPntzkSAztlqE,10,0.643,0.512,11,-8.642,1,0.0308,...,0.000058,0.2740,0.308,137.977,audio_features,spotify:track:4cLcNowHHKPntzkSAztlqE,https://api.spotify.com/v1/tracks/4cLcNowHHKPn...,https://api.spotify.com/v1/audio-analysis/4cLc...,207697,4
4,MALIQ & D'Essentials,Bagaimana Kutahu,22UgCquORz262duHcMTSfM,42,0.602,0.666,3,-7.047,0,0.0631,...,0.000000,0.0911,0.723,84.997,audio_features,spotify:track:22UgCquORz262duHcMTSfM,https://api.spotify.com/v1/tracks/22UgCquORz26...,https://api.spotify.com/v1/audio-analysis/22Ug...,172235,4


In [51]:
df6.to_csv('indienesia_playlist.csv')

In [76]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'indonesia'

for i in range(0,1000,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df7 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df7.head()

Time to run this code (in seconds): 6.87805249999974
(1000, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.856951499999923
Shape of the dataset: (1000, 18)
Shape of the dataset: (1000, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,NIKI,lowkey,5TTXEcfsYLh6fTarLaevTi,76,0.752,0.494,0,-8.180,1,0.0559,...,0.000000,0.0869,0.227,117.901,audio_features,spotify:track:5TTXEcfsYLh6fTarLaevTi,https://api.spotify.com/v1/tracks/5TTXEcfsYLh6...,https://api.spotify.com/v1/audio-analysis/5TTX...,171356,4
1,Rich Brian,Gospel,2PqSNVmiEQpFkLoNPoDfwI,68,0.681,0.659,9,-6.405,1,0.2680,...,0.000008,0.1100,0.225,156.004,audio_features,spotify:track:2PqSNVmiEQpFkLoNPoDfwI,https://api.spotify.com/v1/tracks/2PqSNVmiEQpF...,https://api.spotify.com/v1/audio-analysis/2PqS...,173294,4
2,NIKI,I Like U,6f7BE3VcZScJx6n3wsf5Z3,70,0.480,0.526,5,-7.099,1,0.2750,...,0.000000,0.0879,0.374,199.662,audio_features,spotify:track:6f7BE3VcZScJx6n3wsf5Z3,https://api.spotify.com/v1/tracks/6f7BE3VcZScJ...,https://api.spotify.com/v1/audio-analysis/6f7B...,267733,3
3,Rich Brian,Drive Safe,6eDfe957aV4sqGnDhlSikJ,66,0.635,0.312,7,-8.134,1,0.0265,...,0.000000,0.1380,0.414,98.017,audio_features,spotify:track:6eDfe957aV4sqGnDhlSikJ,https://api.spotify.com/v1/tracks/6eDfe957aV4s...,https://api.spotify.com/v1/audio-analysis/6eDf...,240118,4
4,Rich Brian,Act Up,36Ajx3OHw1qOAjS4tJs38V,66,0.609,0.772,9,-4.152,1,0.1190,...,0.000000,0.3210,0.384,78.052,audio_features,spotify:track:36Ajx3OHw1qOAjS4tJs38V,https://api.spotify.com/v1/tracks/36Ajx3OHw1qO...,https://api.spotify.com/v1/audio-analysis/36Aj...,198627,4


In [77]:
df7.to_csv('pop_indonesia_playlist.csv')

# Dance

In [59]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'dance'

for i in range(0,800,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df8 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df8.head()

Time to run this code (in seconds): 5.686907600000268
(800, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.2826162000001204
Shape of the dataset: (800, 18)
Shape of the dataset: (800, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Doja Cat,Need to Know,3Vi5XqYrmQgOYBajMWSvCi,93,0.664,0.609,1,-6.509,1,0.0707,...,0.000000,0.0926,0.194,130.041,audio_features,spotify:track:3Vi5XqYrmQgOYBajMWSvCi,https://api.spotify.com/v1/tracks/3Vi5XqYrmQgO...,https://api.spotify.com/v1/audio-analysis/3Vi5...,210560,4
1,Doja Cat,Woman,6Uj1ctrBOjOas8xZXGqKk4,95,0.824,0.764,5,-4.175,0,0.0854,...,0.002940,0.1170,0.881,107.998,audio_features,spotify:track:6Uj1ctrBOjOas8xZXGqKk4,https://api.spotify.com/v1/tracks/6Uj1ctrBOjOa...,https://api.spotify.com/v1/audio-analysis/6Uj1...,172627,4
2,Bruno Mars,Smokin Out The Window,4iN16F8JtVxG2UTzp3avGl,90,0.627,0.618,2,-8.529,1,0.0437,...,0.000000,0.3510,0.848,82.030,audio_features,spotify:track:4iN16F8JtVxG2UTzp3avGl,https://api.spotify.com/v1/tracks/4iN16F8JtVxG...,https://api.spotify.com/v1/audio-analysis/4iN1...,197443,4
3,Mariah Carey,All I Want for Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,92,0.336,0.627,7,-7.463,1,0.0384,...,0.000000,0.0708,0.350,150.273,audio_features,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4
4,Doja Cat,Kiss Me More (feat. SZA),748mdHapucXQri7IAO8yFK,90,0.762,0.701,8,-3.541,1,0.0286,...,0.000158,0.1230,0.742,110.968,audio_features,spotify:track:748mdHapucXQri7IAO8yFK,https://api.spotify.com/v1/tracks/748mdHapucXQ...,https://api.spotify.com/v1/audio-analysis/748m...,208867,4


In [60]:
df8.to_csv('dance_playlist.csv')

# Rap

In [61]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'rap'

for i in range(0,500,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df9 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df9.head()

Time to run this code (in seconds): 3.148522999999841
(500, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.472863899999993
Shape of the dataset: (500, 18)
Shape of the dataset: (500, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Drake,Knife Talk (with 21 Savage ft. Project Pat),2BcMwX1MPV6ZHP4tUT9uq6,90,0.849,0.424,5,-9.579,0,0.3240,...,0.000000,0.0834,0.153,145.887,audio_features,spotify:track:2BcMwX1MPV6ZHP4tUT9uq6,https://api.spotify.com/v1/tracks/2BcMwX1MPV6Z...,https://api.spotify.com/v1/audio-analysis/2BcM...,242966,4
1,Mac Miller,Loud,5aQmsZCESbhasWW5VARdS8,61,0.649,0.822,1,-5.800,1,0.0398,...,0.000038,0.3620,0.201,140.026,audio_features,spotify:track:5aQmsZCESbhasWW5VARdS8,https://api.spotify.com/v1/tracks/5aQmsZCESbha...,https://api.spotify.com/v1/audio-analysis/5aQm...,171149,4
2,Drake,Way 2 Sexy (with Future & Young Thug),0k1WUmIRnG3xU6fvvDVfRG,91,0.803,0.597,11,-6.035,0,0.1410,...,0.000005,0.3230,0.331,136.008,audio_features,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4
3,Drake,Up All Night,75L0qdzRnhwV62UXoNq3pE,63,0.606,0.811,7,-4.667,1,0.2780,...,0.000015,0.3490,0.276,156.347,audio_features,spotify:track:75L0qdzRnhwV62UXoNq3pE,https://api.spotify.com/v1/tracks/75L0qdzRnhwV...,https://api.spotify.com/v1/audio-analysis/75L0...,234333,4
4,Post Malone,One Right Now (with The Weeknd),00Blm7zeNqgYLPtW6zg8cj,93,0.687,0.781,1,-4.806,1,0.0530,...,0.000000,0.0755,0.688,97.014,audio_features,spotify:track:00Blm7zeNqgYLPtW6zg8cj,https://api.spotify.com/v1/tracks/00Blm7zeNqgY...,https://api.spotify.com/v1/audio-analysis/00Bl...,193507,4


In [62]:
df9.to_csv('rap_playlist.csv')

# Hiphop

In [65]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'k-pop rap'

for i in range(0,500,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df10 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df10.head()

Time to run this code (in seconds): 2.8600263000002997
(307, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.1632810000000973
Shape of the dataset: (307, 18)
Shape of the dataset: (307, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Younha,WINTER FLOWER(Feat.RM),4Eefey3uN90Abhbr47zKmC,66,0.564,0.796,0,-3.698,1,0.0737,...,0.000000,0.0938,0.241,95.946,audio_features,spotify:track:4Eefey3uN90Abhbr47zKmC,https://api.spotify.com/v1/tracks/4Eefey3uN90A...,https://api.spotify.com/v1/audio-analysis/4Eef...,221133,4
1,Primary,U (Feat. Kwon Jin Ah & Rap Monster),5LlgJFQLF4uXmt1nCtdyTJ,44,0.462,0.658,0,-8.679,1,0.2040,...,0.001510,0.1670,0.605,116.987,audio_features,spotify:track:5LlgJFQLF4uXmt1nCtdyTJ,https://api.spotify.com/v1/tracks/5LlgJFQLF4uX...,https://api.spotify.com/v1/audio-analysis/5Llg...,255011,5
2,Monsta X,BESIDE U (I.M RAP VER) (feat. Pitbull),5SJQSRZ7HhG7J2Jf7ezGIE,41,0.693,0.639,2,-7.293,1,0.2910,...,0.000000,0.2590,0.315,179.842,audio_features,spotify:track:5SJQSRZ7HhG7J2Jf7ezGIE,https://api.spotify.com/v1/tracks/5SJQSRZ7HhG7...,https://api.spotify.com/v1/audio-analysis/5SJQ...,212518,4
3,Wonder Girls,Tell Me - Rap Version,0j4amgyatouEP4Ox3qDrYk,40,0.793,0.946,11,-4.923,0,0.0330,...,0.000181,0.2960,0.969,126.991,audio_features,spotify:track:0j4amgyatouEP4Ox3qDrYk,https://api.spotify.com/v1/tracks/0j4amgyatouE...,https://api.spotify.com/v1/audio-analysis/0j4a...,217120,4
4,JIMIN,Puss,3qcTnkyiU6w416H1Yo8Oio,37,0.817,0.936,1,-3.897,1,0.0895,...,0.000090,0.2090,0.864,99.982,audio_features,spotify:track:3qcTnkyiU6w416H1Yo8Oio,https://api.spotify.com/v1/tracks/3qcTnkyiU6w4...,https://api.spotify.com/v1/audio-analysis/3qcT...,182750,4


In [66]:
df10.to_csv('kpop_hiphop.csv')

# Romance

In [68]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'love'

for i in range(0,800,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df11 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df11.head()

Time to run this code (in seconds): 5.474408900000071
(800, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.6164914000000863
Shape of the dataset: (800, 18)
Shape of the dataset: (800, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Chaka Demus & Pliers,Murder She Wrote,1HGyhNaRUFEDBiVLbvtbL6,66,0.883,0.733,11,-9.153,0,0.128,...,0.000260,0.3340,0.947,95.103,audio_features,spotify:track:1HGyhNaRUFEDBiVLbvtbL6,https://api.spotify.com/v1/tracks/1HGyhNaRUFED...,https://api.spotify.com/v1/audio-analysis/1HGy...,245000,4
1,Gyptian,Hold You - Hold Yuh,0j55p8YRAPG8yajtNyjt6A,67,0.690,0.621,8,-3.583,1,0.060,...,0.000028,0.0884,0.497,66.858,audio_features,spotify:track:0j55p8YRAPG8yajtNyjt6A,https://api.spotify.com/v1/tracks/0j55p8YRAPG8...,https://api.spotify.com/v1/audio-analysis/0j55...,233253,3
2,Gregory Isaacs,Night Nurse,4DQttwipnILl88cru3BRZx,64,0.824,0.366,7,-10.960,1,0.145,...,0.106000,0.0403,0.895,73.949,audio_features,spotify:track:4DQttwipnILl88cru3BRZx,https://api.spotify.com/v1/tracks/4DQttwipnILl...,https://api.spotify.com/v1/audio-analysis/4DQt...,249533,4
3,Bunny Wailer,Mellow Mood,6r9KZ2g5dXZXXLF6vOHnet,61,0.730,0.561,0,-12.851,1,0.203,...,0.047300,0.0493,0.691,80.892,audio_features,spotify:track:6r9KZ2g5dXZXXLF6vOHnet,https://api.spotify.com/v1/tracks/6r9KZ2g5dXZX...,https://api.spotify.com/v1/audio-analysis/6r9K...,231733,4
4,Bunny Wailer,Dream Land,7vp89zcbYncoBCbX5HA3tZ,62,0.782,0.430,1,-14.285,1,0.363,...,0.000294,0.0603,0.890,72.339,audio_features,spotify:track:7vp89zcbYncoBCbX5HA3tZ,https://api.spotify.com/v1/tracks/7vp89zcbYnco...,https://api.spotify.com/v1/audio-analysis/7vp8...,166773,4


In [69]:
df11.to_csv('love_playlist.csv')

# Classical

In [73]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'classical'

for i in range(0,500,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df12 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df12.head()

Time to run this code (in seconds): 3.7167256000002453
(500, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.6603759000004175
Shape of the dataset: (500, 18)
Shape of the dataset: (500, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Mykola Dmytrovych Leontovych,Carol of the Bells,4tHqQMWSqmL6YjXwsqthDI,79,0.418,0.1060,8,-22.507,0,0.0448,...,0.0292,0.1790,0.800,46.718,audio_features,spotify:track:4tHqQMWSqmL6YjXwsqthDI,https://api.spotify.com/v1/tracks/4tHqQMWSqmL6...,https://api.spotify.com/v1/audio-analysis/4tHq...,85267,4
1,Ludovico Einaudi,Experience,1BncfTJAWxrsxyT9culBrj,81,0.447,0.4490,2,-10.634,1,0.0376,...,0.9610,0.0697,0.036,92.468,audio_features,spotify:track:1BncfTJAWxrsxyT9culBrj,https://api.spotify.com/v1/tracks/1BncfTJAWxrs...,https://api.spotify.com/v1/audio-analysis/1Bnc...,315427,4
2,Trans-Siberian Orchestra,Christmas Eve / Sarajevo 12/24 - Instrumental,3sBnSBnzpEYKpJYGsAIbup,66,0.292,0.5670,4,-9.487,0,0.0316,...,0.8520,0.0947,0.156,186.405,audio_features,spotify:track:3sBnSBnzpEYKpJYGsAIbup,https://api.spotify.com/v1/tracks/3sBnSBnzpEYK...,https://api.spotify.com/v1/audio-analysis/3sBn...,205333,3
3,Erik Satie,Gymnopédie No. 1,5NGtFXVpXSvwunEIGeviY3,71,0.469,0.0128,7,-36.856,1,0.1190,...,0.9370,0.0941,0.354,72.765,audio_features,spotify:track:5NGtFXVpXSvwunEIGeviY3,https://api.spotify.com/v1/tracks/5NGtFXVpXSvw...,https://api.spotify.com/v1/audio-analysis/5NGt...,205867,3
4,Johann Sebastian Bach,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",61dYvvfIRtIDFuqZypPAta,65,0.257,0.1090,7,-21.356,1,0.0597,...,0.8670,0.0762,0.307,75.020,audio_features,spotify:track:61dYvvfIRtIDFuqZypPAta,https://api.spotify.com/v1/tracks/61dYvvfIRtID...,https://api.spotify.com/v1/audio-analysis/61dY...,151480,4


In [74]:
df12.to_csv('classical_playlist.csv')

# Country music

In [75]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []
genre = 'country music'

for i in range(0,800,50):
    track_results = sp.search(q='genre:'+genre, type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

        
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


#dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

#dataframe audio
df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df13 = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df13.head()

Time to run this code (in seconds): 6.704203000000234
(800, 4)
Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 2.630895199999941
Shape of the dataset: (800, 18)
Shape of the dataset: (800, 18)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,The Doobie Brothers,Listen to the Music,7Ar4G7Ci11gpt6sfH9Cgz5,76,0.645,0.598,1,-13.180,0,0.0335,...,0.000004,0.0721,0.913,105.929,audio_features,spotify:track:7Ar4G7Ci11gpt6sfH9Cgz5,https://api.spotify.com/v1/tracks/7Ar4G7Ci11gp...,https://api.spotify.com/v1/audio-analysis/7Ar4...,227267,4
1,Tim McGraw,Humble And Kind,4Pn0JlCUusD2QHjADuOzuV,69,0.355,0.480,11,-7.310,1,0.0282,...,0.000002,0.1200,0.137,100.956,audio_features,spotify:track:4Pn0JlCUusD2QHjADuOzuV,https://api.spotify.com/v1/tracks/4Pn0JlCUusD2...,https://api.spotify.com/v1/audio-analysis/4Pn0...,259267,3
2,The Band,The Weight - Remastered 2000,0P7DoyGrr4Wp9w5TotEtUC,73,0.630,0.551,9,-9.280,1,0.0549,...,0.000058,0.1030,0.518,143.868,audio_features,spotify:track:0P7DoyGrr4Wp9w5TotEtUC,https://api.spotify.com/v1/tracks/0P7DoyGrr4Wp...,https://api.spotify.com/v1/audio-analysis/0P7D...,274493,4
3,Alabama,Mountain Music,7BjmEKAVTnJNkRNaUzBy2K,61,0.467,0.526,9,-9.982,1,0.0433,...,0.000000,0.1240,0.753,109.555,audio_features,spotify:track:7BjmEKAVTnJNkRNaUzBy2K,https://api.spotify.com/v1/tracks/7BjmEKAVTnJN...,https://api.spotify.com/v1/audio-analysis/7Bjm...,218613,4
4,Brett Eldredge,Beat of the Music,3un2KQUaQ2brEpd8dK93wI,61,0.629,0.808,9,-4.125,1,0.0269,...,0.000009,0.0751,0.773,110.001,audio_features,spotify:track:3un2KQUaQ2brEpd8dK93wI,https://api.spotify.com/v1/tracks/3un2KQUaQ2br...,https://api.spotify.com/v1/audio-analysis/3un2...,179293,4


In [78]:
df13.to_csv('country_music_playlist.csv')